In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import sqlparse
import re
from SQLParser import *
import SQLFunctions as funcs
from SQLParserError import SQLParserError
import warnings

In [2]:
sql = SQLParser()

In [65]:
raw = '''
SELect
                             t.report_date
  ,      t.gendeR
,   active
,sUm(t.vol) as sum_vol
, Count(t.xsdf) as dfv
, avg(sdf) as fpl
, sum(total_days_per_month) over (partition by id order by subs_id rows between 2 preceding and current row) as cum_sum_90d_1
, sum(total_days_per_month) over (partition by subs_id order by subs_id, end_of_month rows between 2 preceding and current row) as cum_sum_90d_2
, sum(total_days_per_month) over (partition by subs_id, id, a, b, c order by subs_id, end_of_month, a, b, x rows between 2 preceding and current row) as cum_sum_90d_3
, sum(total_days_per_month) over (order by subs_id) as cum_sum_90d_4
, sum(total_days_per_month) over (partition by subs_id, id order by subs_id, end_of_month rows between 2 preceding and current row) as cum_sum_90d_5
, buzz
, sum(total_days_per_month) over (partition by subs_id order by subs_id, end_of_month rows between 2 preceding and current row) as cum_sum_90d_6
  , t.gendeR
    , t.gendeR1
      , t.gendeR2
        , t.gendeR3
, sum(total_days_per_month) over (order by subs_id, end_of_month rows between 2 preceding and current row) as cum_sum_90d_7
, foo
, foo1
, foo2
, foo3
, foo4
FROM active_data_users AS t
LEFT JOIN traffic AS trf ON
JOIN table as tab on t.subs_id = tab.subs_id or t.report_date = tab.report_date1_
and (t.subs_id = trf.subs_id or t.id = trf.id)
left join bar on t.num = bar.num and t.dt = bar.dt and (t.foo = tab.foo or t.buzz = tab.buzz)
where  t> 3  and  t=0 and (d>t.wef or d>6 or t.vol = 49 or t.vol = 10 or y =13) and f=  12 or t.vol  =   r.top and i in   (1111, 111222, 2222222222, 22222222444444444, 4444444444444555555555555555555)
GROUP BY
1, 2, 3,4
, t.report_date
, age_group
, t.gender;

select * from tab join buzz on a=b where 1=1 and id != 111 or name in (xyz, abc);
select * from tabl full outer join foo as f on name  =name where t = 9;

select t.var 
from foo as f
join buzz as b
    on (
        f.id=b.id
        and f.dt=b.dt
        and xyz.sdfjhfd = c.asdnweionf
    )
where 1=  1 and b.date >= '2022-10-01' and t.name != 'not in RU'
'''

In [66]:
d = sql.default_string(raw)

In [69]:
def window_functions_sub(raw):
        '''
        Функция исправляет отступы в оконных функциях. Количество отступов можно регулировать. 
        '''
        assert isinstance(raw, str), 'Аргумент raw должен быть типа string'
        
        if ') OVER (' in raw:
            script_by_rows = [tup for tup in raw.split('\n')]
            
            wind_keywords = ['SELECT', 'SELECT']
            wind_end = ['FROM']
            
            wind_groups = []
            counter = 0
            
            # разметка скрипта, выбор только тех частей, где могут быть оконные функции
            for ind in range(0, len(script_by_rows)-1):
                
                # если строка начинается с as_keywords, то запоминаем ее индекс
                if script_by_rows[ind].startswith(tuple(wind_keywords)) and counter == 0:
                    counter = ind
                    
                # если впереди другие операторы (не as_keywords) и счетчик не равен 0, то записываем диапазон
                if script_by_rows[ind+1].startswith(tuple(wind_end)) and counter != 0:
                    wind_groups.append(script_by_rows[counter:ind+1])
                    counter = 0
            
            # в select ищем оконную функцию по ключевой комбинации ") OVER ("
            for group in wind_groups:
                if len(group) > 1:
                    #print(group)
                    counter = 0
                    for ind in range(0, len(group)):
                        if ') OVER (' in group[ind]:
                            counter = ind
                            find_window = group[counter:]
                            break
            
            # Разбиваем оконные функции на группы - каждая группа это оконная функция и столбцы (если есть), 
            # выбранные пользователем, идущие после нее (включая функции)
            only_window = []
            counter = 0
            for j in range(0, len(find_window)-1):
                #print(d[j])
                if ') OVER (' in find_window[j+1]:
                    only_window.append(find_window[counter:j+1])
                    counter = j+1
                    #print(group[ind])
                if j == len(find_window) - 2:
                    #print(d[j+1])
                    only_window.append(find_window[counter:j+2])
            
            # отступы
            func_indent = ' ' * 7
            part_indent = ' ' * 52
            main_sort_indent = ' ' * 39
            post_sort_indent = ' ' * 48
            one_level_sort = ' ' * 26
            other_fields_indent = ' ' * 7
            
            
            for wind in only_window:
                start_flag = 0
                order_flag = 0
                end_of_wind_flag = 0
                for ind in range(0, len(wind)):
                    #print(f'wind: {ind} :', wind[ind])
                    if ')' in wind[ind] or '(' in wind[ind]:
                        re_val = wind[ind].replace(')', '\)').replace('(', '\(')
                        pattern = re.compile(f'{re_val}')
                        right_val = wind[ind].strip()
                    else:
                        re_val = wind[ind]
                        pattern = re.compile(f'{re_val}')
                        right_val = wind[ind].strip()
                    
                    # каждая группа начинается с оконной функции, в одной группе - одна оконка, зажигаем start_flag
                    if ') OVER (' in wind[ind]:
                        right_val = func_indent + wind[ind].strip()
                        raw = pattern.sub(right_val, raw, 1)
                        start_flag = 1
                    
                    # условие для нескольких сортировок
                    if start_flag == 1 and end_of_wind_flag == 0 and order_flag == 1 and ') OVER (' not in wind[ind]:
                        right_val = post_sort_indent + wind[ind].strip()
                        raw = pattern.sub(right_val, raw, 1)
                    
                    # если сортировка по одному полю, то пишем все в одну строчку по умолчанию (можно поставить one_level_sort)
                    if start_flag == 1 and 'ORDER BY' in wind[ind] and ') AS ' in wind[ind]:
                        right_val = main_sort_indent + wind[ind].strip() #one_level_sort
                        raw = pattern.sub(right_val, raw, 1)
                        order_flag = 1
                    
                    # если встретилась сортировка по нескольким полям, то зажигаем order_flag
                    if start_flag == 1 and 'ORDER BY' in wind[ind] and ') AS ' not in wind[ind]:
                        right_val = main_sort_indent + wind[ind].strip()
                        raw = pattern.sub(right_val, raw, 1)
                        order_flag = 1
                    
                    # условие для последней строчки оконной функции с несколькими сортировками
                    if start_flag == 1 and order_flag == 1 and ') AS ' in wind[ind] and '(' not in wind[ind] and 'ORDER BY' not in wind[ind]:
                        right_val = post_sort_indent + wind[ind].strip()
                        raw = pattern.sub(right_val, raw, 1)
                        end_of_wind_flag = 1
                        continue
                    
                    # отступ для полей, которые идут после окончания оконной функции
                    if start_flag == 1 and 'ORDER BY' not in wind[ind] and end_of_wind_flag == 1:
                        re_val = wind[ind].replace(')', '\)').replace('(', '\(')
                        pattern = re.compile(f'{re_val}')
                        right_val = other_fields_indent + wind[ind].strip()
                        raw = pattern.sub(right_val, raw, 1)
                    
                    # условие для нескольких партиций
                    if start_flag == 1 and end_of_wind_flag == 0 and order_flag == 0 and ') OVER (' not in wind[ind]:
                        right_val = post_sort_indent + wind[ind].strip()
                        raw = pattern.sub(right_val, raw, 1)
                        
        return raw#

In [70]:
print(window_functions_sub(d))

['       sum(total_days_per_month) OVER (PARTITION BY id', '                                       ORDER BY subs_id ROWS BETWEEN 2 preceding AND CURRENT ROW) AS cum_sum_90d_1,']
['                                      sum(total_days_per_month) OVER (PARTITION BY subs_id', '                                                                      ORDER BY subs_id,', '                                                                               end_of_month ROWS BETWEEN 2 preceding AND CURRENT ROW) AS cum_sum_90d_2,']
['                                                                     sum(total_days_per_month) OVER (PARTITION BY subs_id,', '                                                                                                                  id,', '                                                                                                                  a,', '                                                                                                                

In [54]:
len('                                       ')

39